In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2015','era_2015.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2015-01-01 ... 2015-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2015-01-01 ... 2015-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2015-01-01 00:00:00 1000.0         49.25    -124.85    64.517494  0.002700
                                            -124.60    60.404217  0.002418
                                            -124.35    61.613201  0.002445
                                            -124.10    64.113197  0.002503
                                            -123.85    63.630779  0.002462
...                                                          ...       ...
2015-12-31 21:00:00 1000.0         24.50    -68.10     86.309616  0.015418
                                            -67.85     86.401413  0.015373
                                            -67.60     86.149460  0.015304
                                            -67.35     85.376022  0.015155
                                            -67.10     84.676804  0.014988

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2015-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2015-12-31 1000.0         49.25    -124.85    81.761909  0.006591
                                   -124.60    79.396706  0.006596
                                   -124.35    77.385742  0.006484
                                   -124.10    74.923225  0.006367
                                   -123.85    71.545448  0.006323
...                                                 ...       ...
                          24.50    -68.10     79.305656  0.015004
                                   -67.85     79.283173  0.014992
                                   -67.60     79.296783  0.014981
                                   -67.35     79.298782  0.014969
                                   -67.10     79.291702  0.014955

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2015-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 81.76 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006591 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2015_county_shapefile','cb_2015_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,005,00161528,0500000US01005,01005,Barbour,06,2291820706,50864677,"POLYGON ((-85.74803 31.61918, -85.74544 31.618..."
1,01,023,00161537,0500000US01023,01023,Choctaw,06,2365954971,19059247,"POLYGON ((-88.47323 31.89386, -88.46888 31.930..."
2,01,035,00161543,0500000US01035,01035,Conecuh,06,2201896058,6643480,"POLYGON ((-87.42720 31.26436, -87.42551 31.268..."
3,01,051,00161551,0500000US01051,01051,Elmore,06,1601876535,99850740,"POLYGON ((-86.41333 32.75059, -86.37115 32.750..."
4,01,065,00161558,0500000US01065,01065,Hale,06,1667804583,32525874,"POLYGON ((-87.87046 32.76244, -87.86818 32.765..."
...,...,...,...,...,...,...,...,...,...,...
3228,45,019,01252740,0500000US45019,45019,Charleston,06,2372842394,1144346152,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ..."
3229,45,077,01248015,0500000US45077,45077,Pickens,06,1285536060,40612589,"MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ..."
3230,46,123,01265784,0500000US46123,46123,Tripp,06,4176233698,13272785,"POLYGON ((-100.23091 43.49989, -100.23044 43.5..."
3231,47,073,01639752,0500000US47073,47073,Hawkins,06,1261443215,32545400,"POLYGON ((-83.28890 36.37879, -83.28250 36.382..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-85.74803 31.61918, -85.74544 31.618...
1       POLYGON ((-88.47323 31.89386, -88.46888 31.930...
2       POLYGON ((-87.42720 31.26436, -87.42551 31.268...
3       POLYGON ((-86.41333 32.75059, -86.37115 32.750...
4       POLYGON ((-87.87046 32.76244, -87.86818 32.765...
                              ...                        
3228    MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...
3229    MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ...
3230    POLYGON ((-100.23091 43.49989, -100.23044 43.5...
3231    POLYGON ((-83.28890 36.37879, -83.28250 36.382...
3232    POLYGON ((-101.62940 34.75006, -101.62806 34.8...
Name: geometry, Length: 3233, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -85.748032  31.619181
     1   -85.745435  31.618898
     2   -85.742651  31.621259
     3   -85.741740  31.619403
     4   -85.739813  31.621810
...             ...        ...
3232 29 -101.385871  34.748377
     30 -101.434904  34.747420
     31 -101.471562  34.747462
     32 -101.629257  34.747649
     33 -101.629396  34.750056

[1037334 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3232 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 5min 50s
Wall time: 5min 56s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,70.569216,0.010768
1,67.431513,0.01027
2,69.952575,0.010856
3,71.120712,0.010475
4,67.373603,0.010077
...,...,...
3228,73.649967,0.011171
3229,72.925717,0.008941
3230,61.843823,0.006135
3231,72.083236,0.00859


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
AFFGEOID      object
GEOID         object
NAME          object
LSAD          object
ALAND          int64
AWATER         int64
geometry    geometry
fips           int32
dtype: object

In [15]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065
...,...,...,...
3228,Charleston,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...",45019
3229,Pickens,"MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ...",45077
3230,Tripp,"POLYGON ((-100.23091 43.49989, -100.23044 43.5...",46123
3231,Hawkins,"POLYGON ((-83.28890 36.37879, -83.28250 36.382...",47073


In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005,70.569216,0.010768
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,67.431513,0.01027
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035,69.952575,0.010856
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051,71.120712,0.010475
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065,67.373603,0.010077
...,...,...,...,...,...
3228,Charleston,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...",45019,73.649967,0.011171
3229,Pickens,"MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ...",45077,72.925717,0.008941
3230,Tripp,"POLYGON ((-100.23091 43.49989, -100.23044 43.5...",46123,61.843823,0.006135
3231,Hawkins,"POLYGON ((-83.28890 36.37879, -83.28250 36.382...",47073,72.083236,0.00859


In [17]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005,70.569216,0.010768
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,67.431513,0.01027
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035,69.952575,0.010856
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051,71.120712,0.010475
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065,67.373603,0.010077
...,...,...,...,...,...
3228,Charleston,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...",45019,73.649967,0.011171
3229,Pickens,"MULTIPOLYGON (((-82.86687 34.61742, -82.86451 ...",45077,72.925717,0.008941
3230,Tripp,"POLYGON ((-100.23091 43.49989, -100.23044 43.5...",46123,61.843823,0.006135
3231,Hawkins,"POLYGON ((-83.28890 36.37879, -83.28250 36.382...",47073,72.083236,0.00859


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Barbour,1005,70.569216240401147,0.010767536684148
1,Choctaw,1023,67.431512758018741,0.010269540413880
2,Conecuh,1035,69.952574972219850,0.010856360232131
3,Elmore,1051,71.120712122262503,0.010475063244589
4,Hale,1065,67.373603257950478,0.010077094249350
...,...,...,...,...
3228,Charleston,45019,73.649967095491377,0.011170869795444
3229,Pickens,45077,72.925717018801038,0.008941088885408
3230,Tripp,46123,61.843822604546077,0.006134677038488
3231,Hawkins,47073,72.083235861859094,0.008590280044645


In [20]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
1807,Autauga,1001,69.260947459983470,0.010353495891824
3114,Baldwin,1003,74.694437550587693,0.012146967984994
0,Barbour,1005,70.569216240401147,0.010767536684148
2268,Bibb,1007,70.430036743670925,0.010215241320444
3039,Blount,1009,70.977452873871812,0.009800986407162
...,...,...,...,...
2267,Sweetwater,56037,56.034595018139584,0.004004865292238
288,Teton,56039,67.136797087391059,0.004031492022852
1618,Uinta,56041,56.873968197859867,0.003895890448160
1619,Washakie,56043,54.247966317572569,0.004305868318690


In [21]:
county_var.to_pickle('era_2015.pkl')